In [ ]:
import pandas as pd
import numpy as np
import csv


In [6]:
market_price = pd.read_csv(r"C:\Users\DELL\DATASCIENCE\market.csv")
standard_price = pd.read_csv(r"C:\Users\DELL\DATASCIENCE\standard.csv")
coc = pd.read_csv(r"C:\Users\DELL\DATASCIENCE\cost-of-cultivation.csv")


In [7]:
products = ["groundnut", "paddy", "rice", "wheat", "barley", 
            "jowar", "bajra", "maize", "ragi", "gram", "tur", 
            "mustard", "soyabean", "sunflower", 
            "cotton", "jute", "sugarcane"]
market_edited = market_price[market_price['commodity'].str.lower().isin([x.lower() for x in products])]

In [8]:
#take mean of all varieties of one crop
grouped_market = market_edited.groupby(['state', 'commodity']).agg(['mean']).reset_index()
grouped_market.columns = ["".join(x) for x in grouped_market.columns.ravel()]

In [9]:
#convert price in quintal to price in kg
grouped_market['modal_pricemean'] = grouped_market['modal_pricemean'].apply(lambda x: float(x)/100)
standard_price['price'] = standard_price['price'].apply(lambda x: float(x)/100)

In [10]:
#convert to dictionary of crop to profit or cost
std_prices = dict(zip(standard_price.standard, standard_price.price))
coc_dict = dict(zip(coc.crop, coc.cost))

In [11]:
#state vs crop matrix
yield_mat = pd.read_csv(r"C:\Users\DELL\DATASCIENCE\state-crop-yield.csv")

#state and crop lists
states = yield_mat['state']
crops = list(yield_mat.columns)[1:]

In [12]:
yield_np_mat = yield_mat.as_matrix()

prediction_table = list()
prediction_table.append(["state", "crop", "profit"])

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [13]:
for i in range(0,yield_np_mat.shape[0]-1):
    for j in range(1,yield_np_mat.shape[1]):
        if(yield_np_mat[i][j]!=0.0):
	    #if yielding in current state then calculate profit
            rs_per_kg = grouped_market[(grouped_market['state']==states[i]) & (grouped_market['commodity'] == crops[j-1])]['modal_pricemean']
            if(rs_per_kg.shape[0]==1):
		#use market price data
                diff = ((yield_np_mat[i][j]) * rs_per_kg.values[0]) - coc_dict[str(crops[j-1].lower())]
                val = diff if diff>0.0 else 1.0
            else:
		#use approximated standard price
                diff = ((yield_np_mat[i][j]) * std_prices[str(crops[j-1].lower())]) - coc_dict[str(crops[j-1].lower())]
                val = diff if diff>0.0 else 1.0
            
	    #append all results to a table
            prediction_table.append([states[i], crops[j-1], val])
        else:
	    #states which cant produce the given crop
            print("**", states[i],crops[j-1])
            prediction_table.append([states[i], crops[j-1], -1])

** Andhra Pradesh Barley
** Andhra Pradesh Jute
** Arunachal Pradesh  Jowar
** Arunachal Pradesh  Bajra
** Arunachal Pradesh  Ragi
** Arunachal Pradesh  Barley
** Arunachal Pradesh  Gram
** Arunachal Pradesh  Groundnut
** Arunachal Pradesh  Sunflower
** Arunachal Pradesh  Cotton
** Arunachal Pradesh  Jute
** Arunachal Pradesh  Mesta
** Assam Jowar
** Assam Bajra
** Assam Ragi
** Assam Barley
** Assam Groundnut
** Assam Soyabean
** Assam Sunflower
** Bihar Soyabean
** Bihar Cotton
** Chattisgarh Bajra
** Chattisgarh Jute
** Goa Jowar
** Goa Bajra
** Goa Maize
** Goa Wheat
** Goa Barley
** Goa Gram
** Goa Tur
** Goa Mustard
** Goa Soyabean
** Goa Sunflower
** Goa Cotton
** Goa Jute
** Goa Mesta
** Gujarat Barley
** Gujarat Sunflower
** Gujarat Jute
** Gujarat Mesta
** Haryana Ragi
** Haryana Soyabean
** Haryana Jute
** Haryana Mesta
** Himachal Pradesh Jowar
** Himachal Pradesh Bajra
** Himachal Pradesh Tur
** Himachal Pradesh Groundnut
** Himachal Pradesh Sunflower
** Himachal Pradesh C

In [14]:
#print(prediction_table)

#save to csv
with open("state-profit-data.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_table)